# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Check Convergence

### Overparameterization

In [2]:
n = 3
d = 2**n
num_reps = 10
num_iter = 4000

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs, _ = generate_pauli_circuits(n = n, 
                                    circuit_target=None, 
                                    N = None, 
                                    trace=False)

rank = 3
loss_values = np.zeros((num_iter, num_reps))
c_values = np.zeros((num_iter, num_reps))


for i in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)

    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    c = random.uniform(0,1)
    kraus_model = KrausMap(
                       U=U, 
                       c=c, 
                       d=d, 
                       rank=d**2-1
                      )

    model = ModelQuantumMap(
                           q_map = kraus_model,
                           loss = probs_loss,
                           optimizer = tf.optimizers.Adam(learning_rate=0.01),
                           )


    model.train(inputs = inputs,
                targets = targets,
                inputs_val = None,
                targets_val = [kraus_target],
                num_iter = num_iter,
                N = 500,
                verbose = False,
                )
    
    loss_values[:, i] = np.array(model.loss_list)
    c_values[:, i] = np.array(model.c_list)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [3]:
data = [loss_values, c_values]
pickle.dump(data, open("../../data/compressed_sensing_overparam.p", "wb"))

### Under-parameterization

In [15]:
n = 3
d = 2**n
num_reps = 10
num_iter = 4000

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs, _ = generate_pauli_circuits(n = n, 
                                    circuit_target=None, 
                                    N = None, 
                                    trace=False)

rank = 3
loss_values = np.zeros((num_iter, num_reps))
c_values = np.zeros((num_iter, num_reps))


for i in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank=7)

    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    c = random.uniform(0,1)
    kraus_model = KrausMap(
                       U=U, 
                       c=c, 
                       d=d, 
                       rank=rank
                      )

    model = ModelQuantumMap(
                           q_map = kraus_model,
                           loss = probs_loss,
                           optimizer = tf.optimizers.Adam(learning_rate=0.01),
                           )


    model.train(inputs = inputs,
                targets = targets,
                inputs_val = None,
                targets_val = [kraus_target],
                num_iter = num_iter,
                N = 500,
                verbose = False,
                )
    
    loss_values[:, i] = np.array(model.loss_list)
    c_values[:, i] = np.array(model.c_list)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
data = [loss_values, c_values]
pickle.dump(data, open("../../data/compressed_sensing_underparam.p", "wb"))

## Sparse Input
### Generate Data

In [6]:
np.random.seed(42)
random.seed(42)

N_list = [int(16*2**(i/4)) for i in range(21)]
num_reps = 10

### Low Rank

In [7]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 3

prior_fidelity = np.zeros((len(N_list), num_reps))
noPrior_fidelity = np.zeros((len(N_list), num_reps))

for rep in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)
    
    inputs, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = None, 
                                        trace=False)
    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    for i, N in enumerate(N_list):
        kraus_model = KrausMap(
                               U=U, 
                               c=0.9, 
                               d=d, 
                               rank=rank,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        prior_fidelity[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

        kraus_model = KrausMap(
    #                           U=U, 
    #                           c=0.9, 
                               d=d, 
                               rank=rank+1,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        noPrior_fidelity[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [8]:
data = [prior_fidelity, noPrior_fidelity]
pickle.dump(data, open("../../data/compressed_sensing_rank4.p", "wb"))

### Medium Rank

In [9]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 7

prior_fidelity2 = np.zeros((len(N_list), num_reps))
noPrior_fidelity2 = np.zeros((len(N_list), num_reps))

for rep in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)
    
    inputs, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = None, 
                                        trace=False)
    
    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    for i, N in enumerate(N_list):
        kraus_model = KrausMap(
                               U=U, 
                               c=0.9, 
                               d=d, 
                               rank=rank,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        prior_fidelity2[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

        kraus_model = KrausMap(
    #                           U=U, 
    #                           c=0.9, 
                               d=d, 
                               rank=rank+1,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        noPrior_fidelity2[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [10]:
data = [prior_fidelity2, noPrior_fidelity2]
pickle.dump(data, open("../../data/compressed_sensing_rank8.p", "wb"))

### High Rank

In [11]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 15

prior_fidelity3 = np.zeros((len(N_list), num_reps))
noPrior_fidelity3 = np.zeros((len(N_list), num_reps))

for rep in tqdm(range(num_reps)):
    X, _, _ = generate_ginibre(d, d)
    U = generate_unitary(X) # Random unitary part
    kraus_target = KrausMap(U, 0.5, d, rank)
    
    inputs, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = None, 
                                        trace=False)
    
    states = kraus_target.apply_map(inputs[0])
    targets = measurement(states, U_basis = inputs[1])
    
    for i, N in enumerate(N_list):
        kraus_model = KrausMap(
                               U=U, 
                               c=0.9, 
                               d=d, 
                               rank=rank,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        prior_fidelity3[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

        kraus_model = KrausMap(
    #                           U=U, 
    #                           c=0.9, 
                               d=d, 
                               rank=rank+1,
                              )

        model = ModelQuantumMap(
                                q_map = kraus_model,
                                loss = probs_loss,
                                optimizer = tf.optimizers.Adam(learning_rate=0.01),
                                )

        model.train(inputs = [inputs[0][:N], inputs[1][:N]],
                    targets = targets[:N],
                    inputs_val = None,
                    targets_val = None,
                    num_iter = 4000,
                    N = N,
                    verbose = False,
                    )

        noPrior_fidelity3[i, rep] = np.abs(channel_fidelity(kraus_target, kraus_model).numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [ ]:
data = [prior_fidelity3, noPrior_fidelity3]
pickle.dump(data, open("../../data/compressed_sensing_rank16.p", "wb"))

## SPAM
### Generate True Model

In [2]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 2
d = 2**n
rank = d**2
c1 = 0.9
c2 = 0.9

#prep error and full POVM error
spam_target = SPAM(d=d, 
                   use_corr_mat=True
                  )


init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm
spam_target = SPAM(d=d,
                  init = init_target,
                  povm = povm_target)

kraus_target = KrausMap(
                        U=generate_unitary(d=d), 
                        c=0.5, 
                        d=d, 
                        rank=rank-1,
                        spam = spam_target
                        )

### Generate Synthetic Data

In [3]:
inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

In [4]:
inputs_map, _ = generate_pauli_circuits(n = n, 
                                    circuit_target=None, 
                                    N = None, 
                                    trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_map(state)
targets_map = measurement(state, U_basis, spam_target.povm)

### Fit Model

In [5]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

In [6]:
spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 1000,
                    )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.15998353120626813
1 0.15702991200102656
2 0.15423171756701556
3 0.15157822430658352
4 0.14905901544097674
5 0.14666374302834473
6 0.14438367559713647
7 0.14221020052579827
8 0.14013525346164454
9 0.13815129606533835
10 0.13624992092701352
11 0.13442485959683756
12 0.1326707095780096
13 0.13098171715750556
14 0.12935252597795577
15 0.12777814701537432
16 0.12625108022337636
17 0.12476933500889979
18 0.12332881482067279
19 0.12192400609950747
20 0.12055294102908096
21 0.11921209273663888
22 0.11789810991587439
23 0.11660781704311873
24 0.11533822964763216
25 0.11408658221072253
26 0.11284882574816854
27 0.1116243222291915
28 0.11041110195440508
29 0.10920141752334503
30 0.10799935262293228
31 0.10680376517467013
32 0.1056122787239099
33 0.10442566548804352
34 0.1032432484506747
35 0.10206447266528197
36 0.10088590072568852
37 0.09970860126294898
38 0.09853414194123286
39 0.09736235229911781
40 0.09619313581745247
41 0.0950264647016938
42 0.09386237500074707
43 0.09270096194942966
44 

337 0.00036724169409629344
338 0.000363434400088642
339 0.0003596318120473176
340 0.0003558805114148278
341 0.0003523782233668304
342 0.0003488644200618927
343 0.00034546670613757723
344 0.00034219254990624506
345 0.00033899352202208815
346 0.00033586187739961365
347 0.00033283955092971746
348 0.00032985988545985805
349 0.00032691304637985196
350 0.0003241438614417101
351 0.0003213656300486311
352 0.0003186215289083102
353 0.00031588025837726527
354 0.0003132541984099091
355 0.00031068394110700967
356 0.0003081378425583224
357 0.0003055997891118546
358 0.00030308113277687177
359 0.0003006595704427011
360 0.0002982369272551962
361 0.00029585508952842827
362 0.00029340309892972555
363 0.0002909987904889822
364 0.00028864578640258294
365 0.0002863585903457125
366 0.00028410869015257976
367 0.00028189225775773215
368 0.0002797654401997402
369 0.0002776580218739546
370 0.00027558650341125244
371 0.00027353512385804084
372 0.00027154227774610774
373 0.00026958911092805524
374 0.0002676600704

647 6.129404792452263e-05
648 6.103081895764116e-05
649 6.0754381802556715e-05
650 6.048133080847444e-05
651 6.021168391930203e-05
652 5.9945400667968255e-05
653 5.968238659216712e-05
654 5.942249977971449e-05
655 5.916555883253444e-05
656 5.891135162401406e-05
657 5.865964431623305e-05
658 5.841019019719885e-05
659 5.816273798999841e-05
660 5.791703937287282e-05
661 5.767285552940602e-05
662 5.7415831833819584e-05
663 5.7162041138032135e-05
664 5.691146603141419e-05
665 5.666403885488177e-05
666 5.64196449753883e-05
667 5.617812817160636e-05
668 5.593929747189328e-05
669 5.5702934859726466e-05
670 5.546880334259466e-05
671 5.5236654963913235e-05
672 5.500623842044233e-05
673 5.47773060271625e-05
674 5.454961984542579e-05
675 5.431002701571368e-05
676 5.407332766560737e-05
677 5.383954303884967e-05
678 5.360864220557507e-05
679 5.3380545730058895e-05
680 5.3155131358511696e-05
681 5.293224107046025e-05
682 5.2711688913056996e-05
683 5.249326912030217e-05
684 5.227676410455608e-05
685 5

958 1.9139191307175048e-05
959 1.9077018503243836e-05
960 1.9015591986796436e-05
961 1.8954865527132718e-05
962 1.8894777778642282e-05
963 1.8835256131394626e-05
964 1.8776220672344528e-05
965 1.871758803007109e-05
966 1.8659274924000515e-05
967 1.8597620950737787e-05
968 1.853673188823965e-05
969 1.847661335695718e-05
970 1.8417249591354322e-05
971 1.8358604954024086e-05
972 1.830062647773309e-05
973 1.8243247104949886e-05
974 1.818638932396752e-05
975 1.8129968938572974e-05
976 1.807389875114954e-05
977 1.801809198414183e-05
978 1.795901230311064e-05
979 1.7900633924988262e-05
980 1.7842974367530056e-05
981 1.7786029016909307e-05
982 1.7729772744766544e-05
983 1.7674162547597175e-05
984 1.7619140871620157e-05
985 1.7564639316904785e-05
986 1.751058245387363e-05
987 1.7456891529876362e-05
988 1.7400374830833387e-05
989 1.7344522403738244e-05
990 1.7289332443978084e-05
991 1.7234785910491868e-05
992 1.718084781563285e-05
993 1.7127469371714754e-05
994 1.7074590715105153e-05
995 1.70221

In [7]:
spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 #inputs_val = input_map_list,
                 #targets_val = targets_map,
                 num_iter = 1000,
                 N = N_spam
                )

  0%|          | 0/1000 [00:00<?, ?it/s]

0 0.05653671635582137
1 0.04870378717681603
2 0.040705707605270425
3 0.03371306556819722
4 0.02812817421970074
5 0.02361846731504076
6 0.019796147492566574
7 0.016585621624044495
8 0.013960637742267058
9 0.011821958722639858
10 0.010062048170585924
11 0.00867244695411773
12 0.00755254995683343
13 0.006667926488307576
14 0.0059666354348274445
15 0.005422722412998203
16 0.005009733115708485
17 0.004664094103748061
18 0.004369124796946926
19 0.00408503028640736
20 0.0038066080161464535
21 0.0035270012781817503
22 0.003248341977871251
23 0.0029862268778712638
24 0.0027399072587252606
25 0.0025127322439375877
26 0.0023039239075787424
27 0.0021101550805354735
28 0.001928444791885831
29 0.0017519502414672595
30 0.0015889867166979275
31 0.0014581254845927618
32 0.001351328708278028
33 0.0012701436147866767
34 0.0012105202254256266
35 0.0011571722688673892
36 0.0011174340651224416
37 0.0010922444503408373
38 0.001078385253899784
39 0.0010686217208063817
40 0.001055103250652813
41 0.001033460029

325 1.3303986604783296e-05
326 1.3125720659961899e-05
327 1.2741543341679002e-05
328 1.2236955818264325e-05
329 1.1796574504600036e-05
330 1.1531082487061847e-05
331 1.1418949073591111e-05
332 1.1364863984211008e-05
333 1.1267355142218861e-05
334 1.1055126076574441e-05
335 1.0722675161982464e-05
336 1.0355738620628214e-05
337 1.0084798735925306e-05
338 9.970927521634917e-06
339 9.938363967439145e-06
340 9.845549092078495e-06
341 9.628425218241387e-06
342 9.35440860819915e-06
343 9.130941583400974e-06
344 8.987805014634428e-06
345 8.869364009906589e-06
346 8.72249583221702e-06
347 8.555054931328827e-06
348 8.402445735655347e-06
349 8.268962362194833e-06
350 8.128470044570414e-06
351 7.97090340522864e-06
352 7.818208985442111e-06
353 7.689234410126477e-06
354 7.57188077172248e-06
355 7.444148773155149e-06
356 7.3081191198979514e-06
357 7.183725121003725e-06
358 7.073882910771931e-06
359 6.956886656601125e-06
360 6.819463559758068e-06
361 6.680632797041769e-06
362 6.568462001674611e-06
36

655 2.1551857661740743e-07
656 2.1380675980131636e-07
657 2.121125473704559e-07
658 2.1043565111163394e-07
659 2.0877583055256934e-07
660 2.0713300641553232e-07
661 2.0550694815599267e-07
662 2.038973652155425e-07
663 2.023041344074318e-07
664 2.0072709993890347e-07
665 1.9916601007988056e-07
666 1.9762063122157535e-07
667 1.9609079292330733e-07
668 1.945763839912122e-07
669 1.9307721130719926e-07
670 1.9159302377058162e-07
671 1.9012368469227633e-07
672 1.886690517224826e-07
673 1.8722888492846182e-07
674 1.8580299914347238e-07
675 1.8439127389252082e-07
676 1.8299355641488797e-07
677 1.816096535539866e-07
678 1.8023938214438692e-07
679 1.7888261399348074e-07
680 1.7753920183489486e-07
681 1.7620893104689855e-07
682 1.7489163857178357e-07
683 1.7358722460234874e-07
684 1.7229553816886053e-07
685 1.7101639908728348e-07
686 1.6974966669649306e-07
687 1.6849521138054691e-07
688 1.6725287192356074e-07
689 1.660224761368372e-07
690 1.6480389654339775e-07
691 1.6359702953828637e-07
692 1.62

981 2.2287089697296195e-08
982 2.2120304292090504e-08
983 2.1954628229015515e-08
984 2.179005519112208e-08
985 2.162657877403866e-08
986 2.1464192614252092e-08
987 2.1302890515991015e-08
988 2.1142666478458338e-08
989 2.098351456028837e-08
990 2.082542871753123e-08
991 2.066840277737698e-08
992 2.05124305279318e-08
993 2.0357505843791098e-08
994 2.0203622781720757e-08
995 2.005077554690019e-08
996 1.98989583369553e-08
997 1.974816523064665e-08
998 1.9598390212997055e-08
999 1.944962727821629e-08


In [8]:
kraus_model = KrausMap(
                       d = d, 
                       rank = rank,
                       spam = spam_model,
                       )

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

In [9]:
model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = inputs_map,
            targets_val = targets_map,
            num_iter = 1000,
            N = 324
            )

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.13127, val: 0.13127
Step:1, train: 0.12884, val: 0.12884
Step:2, train: 0.12644, val: 0.12644
Step:3, train: 0.12406, val: 0.12406
Step:4, train: 0.12171, val: 0.12171
Step:5, train: 0.11937, val: 0.11937
Step:6, train: 0.11706, val: 0.11706
Step:7, train: 0.11477, val: 0.11477
Step:8, train: 0.11250, val: 0.11250
Step:9, train: 0.11025, val: 0.11025
Step:10, train: 0.10803, val: 0.10803
Step:11, train: 0.10583, val: 0.10583
Step:12, train: 0.10365, val: 0.10365
Step:13, train: 0.10150, val: 0.10150
Step:14, train: 0.09937, val: 0.09937
Step:15, train: 0.09726, val: 0.09726
Step:16, train: 0.09518, val: 0.09518
Step:17, train: 0.09313, val: 0.09313
Step:18, train: 0.09110, val: 0.09110
Step:19, train: 0.08909, val: 0.08909
Step:20, train: 0.08711, val: 0.08711
Step:21, train: 0.08515, val: 0.08515
Step:22, train: 0.08321, val: 0.08321
Step:23, train: 0.08130, val: 0.08130
Step:24, train: 0.07940, val: 0.07940
Step:25, train: 0.07752, val: 0.07752
Step:26, train: 0.0756

Step:217, train: 0.00120, val: 0.00120
Step:218, train: 0.00119, val: 0.00119
Step:219, train: 0.00118, val: 0.00118
Step:220, train: 0.00117, val: 0.00117
Step:221, train: 0.00116, val: 0.00116
Step:222, train: 0.00114, val: 0.00114
Step:223, train: 0.00113, val: 0.00113
Step:224, train: 0.00112, val: 0.00112
Step:225, train: 0.00111, val: 0.00111
Step:226, train: 0.00110, val: 0.00110
Step:227, train: 0.00109, val: 0.00109
Step:228, train: 0.00108, val: 0.00108
Step:229, train: 0.00108, val: 0.00108
Step:230, train: 0.00107, val: 0.00107
Step:231, train: 0.00106, val: 0.00106
Step:232, train: 0.00105, val: 0.00105
Step:233, train: 0.00104, val: 0.00104
Step:234, train: 0.00103, val: 0.00103
Step:235, train: 0.00102, val: 0.00102
Step:236, train: 0.00101, val: 0.00101
Step:237, train: 0.00100, val: 0.00100
Step:238, train: 0.00099, val: 0.00099
Step:239, train: 0.00098, val: 0.00098
Step:240, train: 0.00098, val: 0.00098
Step:241, train: 0.00097, val: 0.00097
Step:242, train: 0.00096,

Step:431, train: 0.00023, val: 0.00023
Step:432, train: 0.00023, val: 0.00023
Step:433, train: 0.00023, val: 0.00023
Step:434, train: 0.00023, val: 0.00023
Step:435, train: 0.00023, val: 0.00023
Step:436, train: 0.00023, val: 0.00023
Step:437, train: 0.00023, val: 0.00023
Step:438, train: 0.00022, val: 0.00022
Step:439, train: 0.00022, val: 0.00022
Step:440, train: 0.00022, val: 0.00022
Step:441, train: 0.00022, val: 0.00022
Step:442, train: 0.00022, val: 0.00022
Step:443, train: 0.00022, val: 0.00022
Step:444, train: 0.00021, val: 0.00021
Step:445, train: 0.00021, val: 0.00021
Step:446, train: 0.00021, val: 0.00021
Step:447, train: 0.00021, val: 0.00021
Step:448, train: 0.00021, val: 0.00021
Step:449, train: 0.00021, val: 0.00021
Step:450, train: 0.00021, val: 0.00021
Step:451, train: 0.00021, val: 0.00021
Step:452, train: 0.00020, val: 0.00020
Step:453, train: 0.00020, val: 0.00020
Step:454, train: 0.00020, val: 0.00020
Step:455, train: 0.00020, val: 0.00020
Step:456, train: 0.00020,

Step:648, train: 0.00006, val: 0.00006
Step:649, train: 0.00006, val: 0.00006
Step:650, train: 0.00006, val: 0.00006
Step:651, train: 0.00006, val: 0.00006
Step:652, train: 0.00006, val: 0.00006
Step:653, train: 0.00006, val: 0.00006
Step:654, train: 0.00006, val: 0.00006
Step:655, train: 0.00006, val: 0.00006
Step:656, train: 0.00006, val: 0.00006
Step:657, train: 0.00006, val: 0.00006
Step:658, train: 0.00006, val: 0.00006
Step:659, train: 0.00006, val: 0.00006
Step:660, train: 0.00006, val: 0.00006
Step:661, train: 0.00006, val: 0.00006
Step:662, train: 0.00006, val: 0.00006
Step:663, train: 0.00006, val: 0.00006
Step:664, train: 0.00006, val: 0.00006
Step:665, train: 0.00006, val: 0.00006
Step:666, train: 0.00006, val: 0.00006
Step:667, train: 0.00005, val: 0.00005
Step:668, train: 0.00005, val: 0.00005
Step:669, train: 0.00005, val: 0.00005
Step:670, train: 0.00005, val: 0.00005
Step:671, train: 0.00005, val: 0.00005
Step:672, train: 0.00005, val: 0.00005
Step:673, train: 0.00005,

Step:865, train: 0.00002, val: 0.00002
Step:866, train: 0.00002, val: 0.00002
Step:867, train: 0.00002, val: 0.00002
Step:868, train: 0.00002, val: 0.00002
Step:869, train: 0.00002, val: 0.00002
Step:870, train: 0.00002, val: 0.00002
Step:871, train: 0.00002, val: 0.00002
Step:872, train: 0.00002, val: 0.00002
Step:873, train: 0.00002, val: 0.00002
Step:874, train: 0.00002, val: 0.00002
Step:875, train: 0.00002, val: 0.00002
Step:876, train: 0.00002, val: 0.00002
Step:877, train: 0.00002, val: 0.00002
Step:878, train: 0.00002, val: 0.00002
Step:879, train: 0.00002, val: 0.00002
Step:880, train: 0.00002, val: 0.00002
Step:881, train: 0.00002, val: 0.00002
Step:882, train: 0.00002, val: 0.00002
Step:883, train: 0.00002, val: 0.00002
Step:884, train: 0.00002, val: 0.00002
Step:885, train: 0.00002, val: 0.00002
Step:886, train: 0.00002, val: 0.00002
Step:887, train: 0.00002, val: 0.00002
Step:888, train: 0.00002, val: 0.00002
Step:889, train: 0.00002, val: 0.00002
Step:890, train: 0.00002,

In [10]:
channel_fidelity(kraus_model, kraus_target)

<tf.Tensor: shape=(), dtype=float64, numpy=0.9952371202477236>